In [1]:
from bs4 import BeautifulSoup 
import numpy as np 
import pandas as pd 
import json 
from geopy.geocoders import Nominatim
import requests 
from pandas.io.json import json_normalize 
from sklearn.cluster import KMeans
import folium # map rendering library
import matplotlib.cm as cm
import matplotlib.colors as colors

print('Libraries imported.')

Libraries imported.


Converion of the website into HTML

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
website_url = requests.get(url).text

In [3]:
soup = BeautifulSoup(website_url,'lxml')

1. Finding the table 
2. Using th to the Headings and td to find the rows
3. Appending them to lists head and i_f

In [4]:
t = soup.find('table',{'class':'wikitable'})
h=t.findAll('th')
head=[]
for t in h:
    hh=str(t)
    head.append(hh[4:-6])
t = soup.find('table',{'class':'wikitable'})
i=t.findAll('td')
items = list(i)
i_f=[]
for i in items:
    ii=str(i)
    i_f.append(ii[4:-6])

In [5]:
rows=np.array(i_f).reshape(int(len(i_f)/3),3)
df = pd.DataFrame(rows,index=None)
df.columns = head
df=df[df.Borough != 'Not assigned'].reset_index()
df['Neighborhood']=df['Neighborhood'].str.replace('/',',')
df=df[['Postal code','Borough',"Neighborhood"]]

In [6]:
df.shape

(103, 3)

In [7]:
d = pd.read_csv('https://cocl.us/Geospatial_data')

In [8]:
d.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [9]:
ll=d[d['Postal Code'].isin(list(df['Postal code']))].set_index('Postal Code')

In [10]:
df.sort_values('Postal code',inplace=True)

In [11]:
dd=df.set_index('Postal code')
dd.rename_axis('postal code',axis='index')
f = dd.join(ll)

In [12]:
f.head(10)

,Borough,Neighborhood,Latitude,Longitude
Postal code,,,,
M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497
M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711
M1G,Scarborough,Woburn,43.770992,-79.216917
M1H,Scarborough,Cedarbrae,43.773136,-79.239476
M1J,Scarborough,Scarborough Village,43.744734,-79.239476
M1K,Scarborough,"Kennedy Park , Ionview , East Birchmount Park",43.727929,-79.262029
M1L,Scarborough,"Golden Mile , Clairlea , Oakridge",43.711112,-79.284577
M1M,Scarborough,"Cliffside , Cliffcrest , Scarborough Village West",43.716316,-79.239476


In [13]:
address = 'Toronto, Ontario Canada'
geolocator = Nominatim(user_agent="foursquare_agent")
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


In [14]:
def makemap(f,n,l):
    mapt = folium.Map(location=[43.6534817, -79.3839347], zoom_start=12)
    # set color scheme for the clusters
    x = np.arange(n)
    ys = [i+x+(i*x)**2 for i in range(n)]
    colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
    rainbow = [colors.rgb2hex(i) for i in colors_array]

    # add markers to the map
    markers_colors = []
    for lat, lon, poi, cluster in zip(f['Latitude'], f['Longitude'],f['Neighborhood'],l):
        label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=rainbow[cluster-1],
            fill=True,
            fill_color=rainbow[cluster-1],
            fill_opacity=0.7).add_to(mapt)

    return  mapt

In [15]:
t_d = f
print(t_d.shape)
t_d.tail()

(103, 4)


,Borough,Neighborhood,Latitude,Longitude
Postal code,,,,
M9N,York,Weston,43.706876,-79.518188
M9P,Etobicoke,Westmount,43.696319,-79.532242
M9R,Etobicoke,"Kingsview Village , St. Phillips , Martin Grov...",43.688905,-79.554724
M9V,Etobicoke,"South Steeles , Silverstone , Humbergate , Jam...",43.739416,-79.588437
M9W,Etobicoke,Northwest,43.706748,-79.594054


In [16]:
CLIENT_ID = 'EEN43PDYEJFHILBS0VAQESWL5TBEHM0ZDFBIAASSI5TUHHQ5' 
CLIENT_SECRET = 'S0JPJP4QKW2WB3NOLOO2S51C35K5AIXRHBEDKDOW4NLEHRUE'
VERSION = '20200404'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: EEN43PDYEJFHILBS0VAQESWL5TBEHM0ZDFBIAASSI5TUHHQ5
CLIENT_SECRET:S0JPJP4QKW2WB3NOLOO2S51C35K5AIXRHBEDKDOW4NLEHRUE


In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [18]:
venues = getNearbyVenues(names=t_d['Neighborhood'],
                                   latitudes=t_d['Latitude'],
                                   longitudes=t_d['Longitude']
                                  )

In [19]:
print('There are {} uniques categories.'.format(len(venues['Venue Category'].unique())))

There are 227 uniques categories.


In [20]:
# one hot encoding
_1h = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
_1h['Neighborhood'] = venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [_1h.columns[-1]] + list(_1h.columns[:-1])
_1h = _1h[fixed_columns]

_1h.head()

,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
group = _1h.groupby('Neighborhood').mean().reset_index()
group.head()

,Neighborhood,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0
1,"Alderwood , Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor , Wilson Heights , Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park , Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.043478,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0


In [22]:
# run k-means clustering
x_train=group.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=10, random_state=1).fit(x_train)

# check cluster labels generated for each row in the dataframe
print(kmeans.labels_[0:10])
print(len(kmeans.labels_))

[6 6 6 0 6 0 0 0 6 6]
95


In [25]:
len(t_d)

103

In [26]:
Final = t_d.head(95)
Final['Label']=kmeans.labels_

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [30]:
Final.tail()

,Borough,Neighborhood,Latitude,Longitude,Label
Postal code,,,,,
M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North",43.653654,-79.506944,6
M8Y,Etobicoke,"Old Mill South , King's Mill Park , Sunnylea ,...",43.636258,-79.498509,7
M8Z,Etobicoke,"Mimico NW , The Queensway West , South of Bloo...",43.628841,-79.520999,6
M9A,Etobicoke,Islington Avenue,43.667856,-79.532242,6
M9B,Etobicoke,"West Deane Park , Princess Gardens , Martin Gr...",43.650943,-79.554724,4


In [31]:
m = makemap(Final,15,kmeans.labels_)

In [32]:
m